In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import cv2
import os
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline

from scipy import pi, cos, sin # used for HoF
from scipy.ndimage import uniform_filter # used for hoF
from numpy.lib.scimath import sqrt # used for hoF
from numpy import arctan2 # used for hoF

from sklearn import svm

import math
import numpy as np
import datetime as dt
from collections import deque
import pandas as pd
 
from sklearn.model_selection import train_test_split
import cv2
from google.colab.patches import cv2_imshow
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Activation, Conv2D, Dense, Dropout, Flatten, MaxPooling2D,BatchNormalization,GlobalAveragePooling2D
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix
from tensorflow.keras.callbacks import ModelCheckpoint,CSVLogger
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix
import tensorflow as tf
# Check GPU coinfiguration in Colab
print("Tensorflow version: ", tf.__version__)
print(tf.test.gpu_device_name())

Tensorflow version:  2.12.0
/device:GPU:0


In [3]:
import os
os.chdir('/content/drive/MyDrive/MTech AI NUS/PM ITSS/')
!ls
directoryName = 'Dataset'

'2D_CNN Model -Diving Dataset.ipynb'
'2D_CNN Model - Flight Dataset.ipynb'
'C3D Model - Diving Dataset.ipynb'
 Dataset
 Dataset_flight
 Flight_Temporal_Segmentation_Final_Pipeline.ipynb
 model_2D1.csv
 model_2D1.h5
 model_2D.csv
 model_2D.h5
 model_c3d.csv
 model_c3d.h5
 model_flightClassify_2D_1.csv
 model_flightClassify_2D_1.h5
 model_flightClassify_2D_2.csv
 model_flightClassify_2D_2.h5
 model_flightClassify_2D_3.csv
 model_flightClassify_2D_3.h5
 model_flightClassify_2D_4.csv
 model_flightClassify_2D_4.h5
 model_flightClassify_2D_5.csv
 model_flightClassify_2D_5.h5
 model_flightClassify_2D.csv
 model_flightClassify_2D.h5
 Temporal_segmentation_final_pipeline.ipynb
 VIDEO


In [4]:
dive_action_labels  = sorted(os.listdir("Dataset"))
print("Dive labels: ", dive_action_labels)

Dive labels:  ['Entry', 'Flight', 'Takeoff']


In [5]:
image_height, image_width = 64, 64

In [6]:
def create_model():
 
    model = Sequential()
 
    model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', input_shape = (image_height, image_width, 3)))
    model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(GlobalAveragePooling2D())
    model.add(Dense(256, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dense(3, activation = 'softmax'))
    model.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['accuracy'])
    # Printing the models summary
    model.summary()
 
    return model

In [7]:
def predict(video_file_path, output_file_path, window_size,model_CNN):
    
    predicted_label_list = []
    predicted_labels_probabilities_deque = deque(maxlen = window_size)
 
    video_reader = cv2.VideoCapture(video_file_path)
 
    original_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_video_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))
 
    video_writer = cv2.VideoWriter(output_file_path, cv2.VideoWriter_fourcc('M', 'P', '4', 'V'), 30, (original_video_width, original_video_height))
 
    while True: 
 
        status, frame = video_reader.read() 
 
        if not status:
            break
 
        resized_frame = cv2.resize(frame, (image_height, image_width))
         
        normalized_frame = resized_frame / 255
 
        predicted_labels_probabilities = model_CNN.predict(np.expand_dims(normalized_frame, axis = 0))[0]
 
        predicted_labels_probabilities_deque.append(predicted_labels_probabilities)
 
        if len(predicted_labels_probabilities_deque) == window_size:
 
            predicted_labels_probabilities_np = np.array(predicted_labels_probabilities_deque)
            print('predicted_labels_probabilities_np : ',predicted_labels_probabilities_np)
 
            predicted_labels_probabilities_averaged = predicted_labels_probabilities_np.mean(axis = 0)
            print('predicted_labels_probabilities_averaged',predicted_labels_probabilities_averaged)
 
            predicted_label = np.argmax(predicted_labels_probabilities_averaged)
            predicted_label_list.append(predicted_label)
 
            predicted_class_name = dive_action_labels[predicted_label]
           
            cv2.putText(frame, predicted_class_name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
 
        video_writer.write(frame)
 
    video_reader.release()
    video_writer.release()
    return predicted_label_list

In [8]:
def get_video_frames(output_video_file_path,predicted_label,output_videoPath):
    video_reader = cv2.VideoCapture(output_video_file_path)
 
    original_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_video_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))
 
    #video_writer = cv2.VideoWriter(output_videoPath, cv2.VideoWriter_fourcc('M', 'P', '4', 'V'), 30, (original_video_width, original_video_height))
    indices_takeoff = [i for i, x in enumerate(predicted_label) if x == 2 ]
    indices_flight = [i for i, x in enumerate(predicted_label) if x == 1]
    indices_entry = [i for i, x in enumerate(predicted_label) if x == 0]
    print(indices_takeoff)
    print(indices_flight)
    print(indices_entry)
    class_list = [indices_entry,indices_flight,indices_takeoff]
    
         
    for i, j in enumerate(class_list):
        video_writer = cv2.VideoWriter(output_videoPath+'_'+str(i)+'.avi', cv2.VideoWriter_fourcc('M', 'P', '4', 'V'), 10, (original_video_width, original_video_height) ) 
        for indices in j:
            video_reader.set(cv2.CAP_PROP_POS_FRAMES,indices)
            #print('Position:', int(video_reader.get(cv2.CAP_PROP_POS_FRAMES)))
            _, frame = video_reader.read()
            video_writer.write(frame)
        #video_reader.release()
        video_writer.release()

    

In [9]:
def create_temporal_segmentation(image_folder,video_filePath,output_video_file_path):
    #image_folder = 'VIDEO/Video_Images/04'
    #video_name = 'VIDEO/video_05.avi'

    images = sorted([img for img in os.listdir(image_folder) if img.endswith(".jpg")])

    frame = cv2.imread(os.path.join(image_folder, images[0]))
    height, width, layers = frame.shape
    print(frame.shape)
    video = cv2.VideoWriter(video_filePath, 0, 30, (width,height))

    for image in images:
        video.write(cv2.imread(os.path.join(image_folder, image)))

    cv2.destroyAllWindows()
    video.release()

    model_CNN = create_model()
    model_CNN.load_weights("model_2D.h5")
    window_size = 3
    predicted_label = predict(video_filePath, output_video_file_path, window_size,model_CNN)
    output_class_folder = video_filePath[:-4]+'_folder'
    get_video_frames(video_filePath,predicted_label,output_class_folder)    

    return predicted_label
  

In [10]:
predicted_label = create_temporal_segmentation('VIDEO/Video_Images/08','VIDEO/video_09.avi','VIDEO/videoClassification_09.avi')
print(predicted_label)

(256, 455, 3)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 64)        1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 60, 60, 64)        36928     
                                                                 
 batch_normalization (BatchN  (None, 60, 60, 64)       256       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 30, 30, 64)       0         
 )                                                               
                                                                 
 global_average_pooling2d (G  (None, 64)               0         
 lobalAveragePooling2D)                                          
                                          